# Movie Recommendation Systeme

In this notebook, we are building a movie recommendation system based on a dataset, found on Kaggle, and presenting Netflix movies and shows.
https://www.kaggle.com/datasets/victorsoeiro/netflix-tv-shows-and-movies?select=titles.csv , Data collected from JustWatch.

For performance purpose, only movies where kept (still: 3744 movies!) - SECTION 1 : Dataset

Finally, the datset include movie title, descriptions, genres, production countries, and imdb and tmdb ratings. Our goal is to create a content-based recommendation system using techniques such as natural language processing (NLP) and machine learning. We will leverage various preprocessing steps, followed by a K-Nearest Neighbors (KNN) algorithm to recommend similar movies based on user preferences.

Key components of this project include:
- Handling Missing Values.
- Cleaning and vectorizing movie descriptions.
- Encoding genres and countries.
- Weighting scores.
- Scaling numerical features such as runtime, release year, ratings.
- Using a KNN algorithm for movie recommendations.

--------------------------
## Dataset Overview

The dataset contains the following important features:
- `id`: A unique identifier for each movie.
- `title`: The title of the movie.
- `description`: A textual description of the movie.
- `genres`: A list of genres associated with the movie.
- `production_countries`: A list of countries where the movie was produced.
- `runtime`: The duration of the movie in minutes.
- `release_year`: The year the movie was released.
- `imdb_score`: The IMDb rating of the movie.
- `imdb_vote` : The number of votes received on the movie.
- `tmdb_score`: The rating of the movie on The Movie Database (TMDb).
- `tmdb_popularity`: Number representing the popularity of the movies, calculate by tmdb.

--------------------------------
## Project Workflow

The workflow of this project can be broken down into several key stages:

1. **Data Preprocessing**:
   - Clean missing values, fill missing countries with `unknown_country`, and predict missing scores using machine learning (Random Forest).
   
2. **Text Processing**:
   - Clean the movie descriptions by removing stopwords, punctuation, and lemmatizing the text to get meaningful tokens.
   - Vectorize the cleaned descriptions using TF-IDF to transform the text into numerical features.

3. **Feature Engineering**:
   - Encode genres and production countries using `MultiLabelBinarizer`.
   - Scale numerical features such as `runtime`, `release_year`, and `my_rating`.

4. **Model Training**:
   - Use the KNN algorithm to recommend movies based on similar features (e.g., description, genres, countries, runtime, release year, and user ratings).

5. **Recommendation System**:
   - Recommend movies based on either a single movie or a group of selected movies, providing personalized suggestions based on user preferences.

In [3]:
# Import necessary libraries
import numpy as np
import pandas as pd
import ast

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestRegressor

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from nltk.stem import WordNetLemmatizer

In [4]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\melan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\melan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\melan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Section 1 : Dataset

In [6]:
# Loading Data
df = pd.read_csv('input/titles.csv')

In [7]:
df.head(3)

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,['documentation'],['US'],1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,"['drama', 'action', 'thriller', 'european']",['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300


In [8]:
# Filtering Movies and Dropping Unnecessary Columns
df = df[df['type']=='MOVIE'].drop(['type','seasons','age_certification','imdb_id'], axis=1).reset_index(drop=True)

In [9]:
df.head(3)

,id,title,description,release_year,runtime,genres,production_countries,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,tm84618,Taxi Driver,A mentally unstable Vietnam War veteran works ...,1976,114,"['drama', 'crime']",['US'],8.2,808582.0,40.965,8.179
1,tm154986,Deliverance,Intent on seeing the Cahulawassee River before...,1972,109,"['drama', 'action', 'thriller', 'european']",['US'],7.7,107673.0,10.010,7.300
2,tm127384,Monty Python and the Holy Grail,"King Arthur, accompanied by his squire, recrui...",1975,91,"['fantasy', 'action', 'comedy']",['GB'],8.2,534486.0,15.461,7.811


In [10]:
len(df)

3744

In [11]:
df['genres'].apply(type).unique()

array([<class 'str'>], dtype=object)

In [12]:
df['genres'] = df['genres'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df['production_countries'] = df['production_countries'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

In [13]:
df['genres'].apply(type).unique()

array([<class 'list'>], dtype=object)

## Section 2 : Class preparation for Pipeline (Missing values, clean text, weighted score)

### Missing Value

Class responsible for handling missing values :  
- Delete lines with no title or description,  
- Replace empty genre and country by unknown,  
- Fill missing score by prediction with **RandomForestRegressor** algorythm 

In [17]:
class MissingValueHandler(BaseEstimator,TransformerMixin):
    def __init__(self):
        pass        
        
    def fit(self, X,y=None):
        return self
    
    def transform(self, X,y=None):
        df = X.copy()
        
        # Les lignes dont title ou description sont vide
        df = df.dropna(subset=['title', 'description'])     
        # Les lignes dont plus de 3 colonnes ont des valeurs nulles
        missing_value_line = df[df.isnull().sum(axis=1) >3]
        df = df.drop(missing_value_line.index)
        
        # Gestions des listes
        # Remplacer les listes vides par des valeurs par défaut
        df['genres'] = df['genres'].apply(lambda x: x if len(x) > 0 else ['unknown'])
        df['production_countries'] = df['production_countries'].apply(lambda x: x if len(x) > 0 else ['unknown_country'])

        # Supprimer les lignes où les deux scores sont manquants
        drop_line = df[(df['imdb_score'].isnull())&(df['tmdb_score'].isnull())]
        df = df.drop(drop_line.index)    
        # Remplir les valeurs manquantes des scores en se basant sur l'autre score
        df['tmdb_score'] = df['tmdb_score'].fillna(df['imdb_score'])
        df['imdb_score'] = df['imdb_score'].fillna(df['tmdb_score'])
        # Predire les valeures manquante des scores  
        def predict_missing_values(df, target_column):
            df_model = df[["release_year", "runtime", "imdb_score", "tmdb_score", target_column]]

            train_data = df_model.dropna(subset=[target_column])
            test_data = df_model[df_model[target_column].isnull()]
            X_train = train_data.drop(target_column, axis=1)
            y_train = train_data[target_column]

            model = RandomForestRegressor(n_estimators=100, random_state=0)
            model.fit(X_train, y_train)

            X_test = test_data.drop(target_column, axis=1)
            predictions = model.predict(X_test)

            df.loc[df[target_column].isnull(), target_column] = predictions
            
        #Appliquer sur les colonnes cible
        target_columns = ['tmdb_popularity','imdb_votes']
        for column in target_columns: 
            predict_missing_values(df, column)
            
        return df            

### Score unique

Class for calculated a weighted score with imdb and tmdb data (that just a math basically...)
Output, in replacement to multiple imdb and tmdb datas, just one rating `my_rating` representing scores and popularity /10.

In [19]:
class ScoreCalculator(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
        
    def fit(self, X,y=None):
        return self
        
    def transform(self, X, y=None):
        df = X.copy()
        
        # Normalizing Scores
        df['tmdb_popularity_norm'] = (df['tmdb_popularity'] - df['tmdb_popularity'].min()) / (df['tmdb_popularity'].max() - df['tmdb_popularity'].min())
        df['imdb_votes_norm'] = (df['imdb_votes'] - df['imdb_votes'].min()) / (df['imdb_votes'].max() - df['imdb_votes'].min())

        # Calculating Weighted Scores
        df['weighted_imdb_score'] = df['imdb_score'] * df['imdb_votes_norm']
        df['weighted_tmdb_score'] = df['tmdb_score'] * df['tmdb_popularity_norm']

        df['my_rating'] = (df['weighted_imdb_score'] + df['weighted_tmdb_score']) / (df['imdb_votes_norm'] + df['tmdb_popularity_norm'])
        
        #drop colonnes : 
        df = df.drop(
            ['tmdb_popularity_norm','tmdb_popularity','tmdb_score','weighted_tmdb_score',
             'imdb_votes','imdb_votes_norm','imdb_score','weighted_imdb_score']
            , axis=1)
        
        return df

### Nettoyer la description

Class responsible for cleaning the movie descriptions.  
It removes stopwords, punctuation and applies lemmatization to the words with **WordNetLemmatizer()**.  
The final output is a cleaned text column that is ready for vectorization.

In [21]:
class TextCleaner(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.stop_words = set(stopwords.words('english'))
        self.punctuation = set(string.punctuation)
        
    def fit(self, X,y=None):
        return self
        
    def transform(self, X, y=None):
        df = X.copy()
        
        words_description  = df['description'].apply(lambda x: x.lower())
        words_description = words_description.apply(word_tokenize)

        # keep words which are not in stopwords and filter punctuation 
        def filter_stop_words(words, stop_words,punctuation):
            filtered_words = []
            for word in words:
                clean_word = ''.join([char for char in word if char not in punctuation ])               
                if clean_word and clean_word.lower() not in stop_words and len(clean_word) > 2:
                    filtered_words.append(clean_word)
            return filtered_words

        # apply filter
        df['description_filtered'] = words_description.apply(lambda x : filter_stop_words(x, self.stop_words, self.punctuation))

        # lemmentization
        lemmatizer = WordNetLemmatizer()        
        def apply_lemmatization(words):           
            lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
            return lemmatized_words 

        df['description_lem'] = df['description_filtered'].apply(lambda x : apply_lemmatization(x))

        # join
        df['description_join'] = df['description_lem'].apply(lambda x : ' '.join(x))
        
        #drop columns 
        df = df.drop(['description_lem','description_filtered'], axis=1)
        
        return df

## Section 3 : Verification df

Not 'really' necessary, the purpose of this section is just to verify the dataset.
Just to be sure, the ground work is funtionnal

In [24]:
pipeline = Pipeline([
    ('missing_values', MissingValueHandler()),
    ('score_calculation', ScoreCalculator()),
    ('text_cleaning', TextCleaner())
])

In [25]:
df_transformed = pipeline.fit_transform(df)

In [26]:
df_transformed.head()

,id,title,description,release_year,runtime,genres,production_countries,my_rating,description_join
0,tm84618,Taxi Driver,A mentally unstable Vietnam War veteran works ...,1976,114,"[drama, crime]",[US],8.198980,mentally unstable vietnam war veteran work nig...
1,tm154986,Deliverance,Intent on seeing the Cahulawassee River before...,1972,109,"[drama, action, thriller, european]",[US],7.665871,intent seeing cahulawassee river turned one hu...
2,tm127384,Monty Python and the Holy Grail,"King Arthur, accompanied by his squire, recrui...",1975,91,"[fantasy, action, comedy]",[GB],8.189007,king arthur accompanied squire recruit knight ...
3,tm120801,The Dirty Dozen,12 American military prisoners in World War II...,1967,150,"[war, action]","[GB, US]",7.677974,american military prisoner world war ordered i...
4,tm70993,Life of Brian,"Brian Cohen is an average young Jewish man, bu...",1979,94,[comedy],[GB],7.991343,brian cohen average young jewish man series ri...


In [27]:
# funtion to verify the empty cells. Output : name columns - %nan (if >0)
def percent_missing(df):
    per_nan = 100*df.isnull().sum() /len(df)
    per_nan = per_nan[per_nan >0].sort_values()    
    return per_nan

In [28]:
percent_missing(df_transformed)

Series([], dtype: float64)

In [29]:
df_transformed.to_csv("input/movie_recom_pandas_vpipeline.csv", index=False) #just to keep a record

## Section 4 : Preprocessing class

### Genre et contry : Binariser genre et country

This class encodes the genres and production countries of the movies. It uses **MultiLabelBinarizer()** to create binary columns for each genre and country, allowing the model to capture these categorical features in a meaningful way.  
Name of the created columns begin by 'genre_' or 'country_' (usefull for features section later)

- **`fit`**: Fits the binarizer on the genres and countries.
- **`transform`**: Transforms the genre and country data into binary columns.


In [66]:
# Extractiondata : multilabelBinarizer
class GenreCountryExtractor(BaseEstimator,TransformerMixin):
        def __init__(self):
            self.genre_binarizer = MultiLabelBinarizer()
            self.country_binarizer = MultiLabelBinarizer()            
        
        def fit(self, X, y=None):
            self.genre_binarizer.fit(X['genres'])
            self.country_binarizer.fit(X['production_countries'])
            return self
        
        def transform(self,X,y=None):
            genre_encoded = self.genre_binarizer.transform(X['genres'])
            country_encoded = self.country_binarizer.transform(X['production_countries'])
            
            genre_df = pd.DataFrame(genre_encoded, index=X.index, columns=[f'genre_{g}' for g in self.genre_binarizer.classes_])
            country_df = pd.DataFrame(country_encoded, index=X.index, columns=[f'country_{c}' for c in self.country_binarizer.classes_])
            
            return pd.concat([X, genre_df, country_df], axis=1)

### Text Vectorization (TF-IDF)

This class vectorizes the cleaned text using **TF-IDF** (Term Frequency-Inverse Document Frequency).
It transforms the cleaned descriptions into numerical vectors, which capture the importance of words in the descriptions.
- **`fit`**: Fits the TF-IDF vectorizer to the descriptions.
- **`transform`**: Transforms the cleaned descriptions into numerical vectors.

In [38]:
class TextVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.vectorizer = TfidfVectorizer(max_features=100, min_df=2, max_df=0.8, token_pattern=r'\b\w\w+\b')
        
    def fit(self, X, y=None):
        self.vectorizer.fit(X['description_join'])
        return self
        
    def transform(self, X,y=None):
        df = X.copy()
        
        tfidf_matrix = self.vectorizer.transform(df['description_join'])
        tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=self.vectorizer.get_feature_names_out(), index=df.index)
        
        # Concaténer le DataFrame d'origine avec les colonnes TF-IDF
        return pd.concat([df, tfidf_df], axis=1)


### Scaling

The FeatureScaler class scales the numerical features (`runtime`, `release_year`, and `my_rating`) using **StandardScaler()**. Scaling is crucial to ensure that these features contribute equally to the distance calculation in the KNN model.  
Reduction of `my_rating` impact. 

- **`fit`**: Learns the scaling parameters from the training data.
- **`transform`**: Scales the numerical features using the learned parameters.

In [129]:
class FeatureScaler(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.scaler = StandardScaler()
        
    def fit(self, X, y=None):
        self.scaler.fit(X[['runtime','release_year','my_rating']])
        return self
        
    def transform(self, X,y=None):
        df = X.copy()
        scaled_values = self.scaler.transform(df[['runtime', 'release_year', 'my_rating']])
        df['runtime_scaled'] = scaled_values[:, 0]
        df['release_year_scaled'] = scaled_values[:, 1]
        df['my_rating_scaled'] = scaled_values[:, 2]
        df['my_rating_scaled'] *= 0.5  # Réduire l'impact de my_rating
        return df

### Combined features

Utility class, usefull for the end of the Pipeline to selected the datas we want to used for the machine learning model. 

In [92]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, feature_prefixes):
        self.feature_prefixes = feature_prefixes
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        selected_columns = []  # Créer une liste vide pour stocker les colonnes sélectionnées
        
        # Parcourir toutes les colonnes de X
        for col in X.columns:
            # Pour chaque colonne, vérifier si elle commence par l'un des préfixes
            for prefix in self.feature_prefixes:
                if col.startswith(prefix):
                    selected_columns.append(col)  # Si oui, ajouter la colonne à la liste
                    
        selected_columns += ['title', 'description', 'genres', 'production_countries', 'my_rating']
        
        # Retourner seulement les colonnes sélectionnées
        return X[selected_columns]

## Section 5 : Pipeline and KNN

### Pipeline

Section 3 was used to "pre-preprocess", as such there is a Pipeline for Missing values, Score calculation and Text cleaning.  
but, you can skip the section 3, and put this 3 process in this one Pipeline.

In [131]:
pipeline = Pipeline([
    ('vectorize_text', TextVectorizer()),  # Vectorized the Clean Text
    ('binarize_genre_country', GenreCountryExtractor()),  # Binarized country and genre
    ('scale_features', FeatureScaler()),  # Scaler runtime, release_year ans rating
    ('select_features', FeatureSelector(feature_prefixes=[
        'runtime_scaled', 'release_year_scaled', 'my_rating_scaled', # Seclection scaled variables
        'genre_', 'country_',  # selection of all country and genre
        'vector_'  # TF-IDF vector
    ]))
])

In [133]:
df_preprocessing = pipeline.fit_transform(df_transformed)

### Implementing KNN

The KNN algorithm is used to recommend movies based on their similarity in terms of features like movie descriptions, genres, countries, runtime, release year, and user ratings. The cosine distance metric is used to measure similarity between movies.

- **Training the KNN model**: The model is trained on the numerical features extracted from the dataset.

In [135]:
df_numeric = df_preprocessing.select_dtypes(include=['float64', 'int64'])

In [137]:
knn = NearestNeighbors(n_neighbors=10, metric='cosine').fit(df_numeric)

## Section 6 : Recommendation

**Making recommendations**: You can recommend movies based on a single movie or a group of movies by calculating the mean vector of the selected movies' features.

In [139]:
film_index = 0
distances, indices = knn.kneighbors(df_numeric.iloc[film_index].values.reshape(1, -1), n_neighbors=10)

C:\Users\melan\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


In [141]:
print("Films recommandés : ", df_preprocessing.iloc[indices[0]][['title', 'description', 'genres', 'production_countries', 'my_rating']])

Films recommandés :                                          title  \
0                                 Taxi Driver   
39                                  Christine   
4                               Life of Brian   
2             Monty Python and the Holy Grail   
1                                 Deliverance   
46                                     Vaashi   
16                                   The Land   
76                          An Egyptian Story   
61                   Adam: His Song Continues   
48  Mobile Suit Gundam II: Soldiers of Sorrow   

                                          description  \
0   A mentally unstable Vietnam War veteran works ...   
39  Geeky student Arnie Cunningham falls for Chris...   
4   Brian Cohen is an average young Jewish man, bu...   
2   King Arthur, accompanied by his squire, recrui...   
1   Intent on seeing the Cahulawassee River before...   
46  Ebin Mathew, a budding lawyer ambitiously join...   
16  Set in 1933, the mayor informs the p

#### With many movies

In [ ]:
# 0 : Taxi Driver
# 120 :
# 38 : Police Academy

In [143]:
film_indices = [0, 120, 38]
mean_vector = np.mean(df_numeric.iloc[film_indices].values, axis=0)
distances, indices = knn.kneighbors(mean_vector.reshape(1, -1), n_neighbors=10)

C:\Users\melan\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


In [145]:
recommended_movies = df_preprocessing.iloc[indices[0]][['title', 'description', 'genres', 'production_countries', 'my_rating']]

In [147]:
recommended_movies

,title,description,genres,production_countries,my_rating
39,Christine,Geeky student Arnie Cunningham falls for Chris...,"[horror, thriller, european]",[US],6.722345
38,Police Academy,New rules enforced by the Lady Mayoress mean t...,"[crime, comedy]",[US],6.682930
62,The George McKenna Story,Washington plays a school principal in a tough...,[drama],[US],6.071166
57,Too Young The Hero,TV movie based upon the true story of Calvin G...,"[drama, war]",[US],6.328117
67,"Alexandria, Again and Forever",Set in 1987 against the backdrop of a hunger s...,[drama],"[FR, EG]",5.851175
59,The Ryan White Story,"The story of Ryan White, a 13-year-old haemoph...",[drama],[US],6.298208
69,Strange Voices,A family begins to fall apart when their eldes...,[drama],[US],6.742445
61,Adam: His Song Continues,Follows the true story of John and Reve Walsh ...,"[crime, drama]",[US],7.000000
46,Vaashi,"Ebin Mathew, a budding lawyer ambitiously join...","[drama, thriller]",[IN],6.700000
173,Nightmare in Columbia County,The recounting of a terrible crime that wracke...,"[drama, thriller, crime]",[US],5.943738
